### Average all depths to model depths 

In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import datetime

In [2]:
names_averaged = []
dfs_averaged = {}

directory = "D:/ИТМО/Python/rozes/parsed_measurements"    
for filename in os.listdir(directory):
    if 'csv' in filename:
        path = f'{directory}/{filename}'
        data = pd.read_csv(path, sep = ';', decimal = ',', index_col = 0)
        df = pd.DataFrame(data)
        name = path.split('/')[-1][:-4]
        names_averaged.append(name)

        df.columns = df.columns.str.replace(' ', '_')
        df.columns = df.columns.str[10:]
        df.columns = df.columns.str[:-7]
        df.fillna(0, inplace=True)
        df.columns.values[0] = "time"
        df.columns.values[1] = "number"
        df.columns.values[2] = "pressure"
        df.columns.values[3] = "temperature"
        
        # average time to the nearest hour 
        df.time = pd.to_datetime(df.time)      
        df.time = df.time.dt.round("H")
        df = df.groupby(['time']).mean()

        # searching columns with depth <= 3.4 m
        list_3 = np.arange(0, 3.5, 0.1).round(1).tolist()
        names_3_Mod = []
        names_3_Dir = []
        for col in df.columns:
            for i in list_3:
                if col[:1] == '_':
                    if f'{i}' in col:
                        if "Mod" in col:
                            names_3_Mod.append(col)
                        elif "Dir" in col:
                            names_3_Dir.append(col)
                elif f'0{i}' in col:
                        if "Mod" in col:
                            names_3_Mod.append(col)
                        elif "Dir" in col:
                            names_3_Dir.append(col)
                elif f'-{i}' in col:
                        if "Mod" in col:
                            names_3_Mod.append(col)
                        elif "Dir" in col:
                            names_3_Dir.append(col)

        # searching columns with 3.4 > depth <= 12.2 m
        list_12 = np.arange(3.5, 12.3, 0.1).round(1).tolist()
        names_12_Mod = []
        names_12_Dir = []
        for col in df.columns:
            for i in list_12:
                    if col[:1] == '_':
                        if f'{i}' in col:
                            if "Mod" in col:
                                names_12_Mod.append(col)
                            elif "Dir" in col:
                                names_12_Dir.append(col)
                    else:
                        if i >= 10:
                            if f'{i}' in col:
                                if "Mod" in col:
                                    names_12_Mod.append(col)
                                elif "Dir" in col:
                                    names_12_Dir.append(col)

        # searching columns with 12.2 > depth <= 25.4 m
        list_25 = np.arange(12.3, 25.5, 0.1).round(1).tolist()
        names_25_Mod = []
        names_25_Dir = []

        for col in df.columns:
            for i in list_25:
                if f'{i}' in col:
                    if "Mod" in col:
                        names_25_Mod.append(col)
                    elif "Dir" in col:
                        names_25_Dir.append(col)

        # searching columns with 25.4 > depth <= 47 m
        list_47 = np.arange(25.5, 47.1, 0.1).round(1).tolist()
        names_47_Mod = []
        names_47_Dir = []

        for col in df.columns:
            for i in list_47:
                if f'{i}' in col:
                    if "Mod" in col:
                            names_47_Mod.append(col)
                    elif "Dir" in col:
                            names_47_Dir.append(col)

        # searching columns depth > 47 m
        list_47_more = np.arange(47.1, 100, 0.1).round(1).tolist()
        names_47_more_Mod = []
        names_47_more_Dir = []

        for col in df.columns:
            for i in list_47_more:
                if f'{i}' in col:
                    if "Mod" in col:
                            names_47_more_Mod.append(col)
                    elif "Dir" in col:
                            names_47_more_Dir.append(col)

        # creating a df with averaged depths
        def averaged_table(df, list, col_name):
            df_avg = df[list]
            df[col_name] = df_avg.mean(axis=1)
            df.drop(list, axis=1, inplace = True)
            
        averaged_table(df, names_3_Mod, "P_Spd_3m")
        averaged_table(df, names_3_Dir, "P_Dir_3m")

        averaged_table(df, names_12_Mod, "P_Spd_12m")
        averaged_table(df, names_12_Dir, "P_Dir_12m")

        averaged_table(df, names_25_Mod, "P_Spd_25m")
        averaged_table(df, names_25_Dir, "P_Dir_25m")

        averaged_table(df, names_47_Mod, "P_Spd_47m")
        averaged_table(df, names_47_Dir, "P_Dir_47m")

        df.drop(names_47_more_Mod, axis=1, inplace = True)
        df.drop(names_47_more_Dir, axis=1, inplace = True)
        df.drop(['number', 'pressure', 'temperature'], axis=1, inplace = True)

        cols = ['P_Spd_47m', 'P_Dir_47m']

        if 'abs05__20-10-17' not in filename: 
            if df[cols].isnull().values.any():
                df.drop(cols, axis=1, inplace = True)
        df = df.fillna(0)

        dfs_averaged[name] = df

        #df.to_csv(f"D:/ИТМО/Python/rozes/parsed_measurements/averaged_depths_(all)/avg_{name}.csv", sep = ';', decimal = ',') 
        #dfs_averaged['abs01__19-10-13']

### Load iceconc  

In [3]:
names_iceconc = []
dfs_iceconc = {}

directory_iceconc = "D:/ИТМО/Python/rozes/iceconc/edited_dates"    
for filename_iceconc in os.listdir(directory_iceconc):
    if 'csv' in filename_iceconc:
        path = f'{directory_iceconc}/{filename_iceconc}'
        data = pd.read_csv(path, sep = ',', decimal = ',', index_col = 0)
        df = pd.DataFrame(data)
        name = path.split('/')[-1][:-4]
        names_iceconc.append(name)
        df.time = pd.to_datetime(df.time)
        dfs_iceconc[name] = df
        #dfs_iceconc['mod05__18-10-21_iceconc']

### Remove iceconc  

In [4]:
for n_iceconc, n_averaged in zip(names_iceconc, names_averaged):
    df_no_iceconc = dfs_iceconc[n_iceconc]
    df_no_iceconc = df_no_iceconc.loc[df_no_iceconc['iceconc-0-value'].astype(float) > 0.5]
    general_name = n_averaged[3:]
    
    df_merge = dfs_averaged[n_averaged].merge(df_no_iceconc, how='left', on='time')  
    df_merge.dropna(inplace=True)
    df_merge.drop('iceconc-0-value', axis = 1, inplace=True)
    df_merge.to_csv(f"D:/ИТМО/Python/rozes/parsed_measurements/averaged_depths_(all)/avg_no_iceconc/avg_no_iceconc{general_name}.csv")